# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

## <font face="仿宋">第四部分导读

&emsp;&emsp;<font face="仿宋">在案例的第二、三部分中，我们详细介绍了关于特征工程的各项技术，特征工程技术按照大类来分可以分为数据预处理、特征衍生、特征筛选三部分，其中特征预处理的目的是为了将数据集整理、清洗到可以建模的程度，具体技术包括缺失值处理、异常值处理、数据重编码等，是建模之前必须对数据进行的处理和操作；而特征衍生和特征筛选则更像是一类优化手段，能够帮助模型突破当前数据集建模的效果上界。并且我们在第二部分完整详细的介绍机器学习可解释性模型的训练、优化和解释方法，也就是逻辑回归和决策树模型。并且此前我们也一直以这两种算法为主，来进行各个部分的模型测试。

&emsp;&emsp;<font face="仿宋">而第四部分，我们将开始介绍集成学习的训练和优化的实战技巧，尽管从可解释性角度来说，集成学习的可解释性并不如逻辑回归和决策树，但在大多数建模场景下，集成学习都将获得一个更好的预测结果，这也是目前效果优先的建模场景下最常使用的算法。

&emsp;&emsp;<font face="仿宋">总的来说，本部分内容只有一个目标，那就是借助各类优化方法，抵达每个主流集成学习的效果上界。换而言之，本部分我们将围绕单模优化策略展开详细的探讨，涉及到的具体集成学习包括随机森林、XGBoost、LightGBM、和CatBoost等目前最主流的集成学习算法，而具体的优化策略则包括超参数优化器的使用、特征衍生和筛选方法的使用、单模型自融合方法的使用，这些优化方法也是截至目前，提升单模效果最前沿、最有效、同时也是最复杂的方法。其中有很多较为艰深的理论，也有很多是经验之谈，但无论如何，我们希望能够围绕当前数据集，让每个集成学习算法优化到极限。值得注意的是，在这个过程中，我们会将此前介绍的特征衍生和特征筛选视作是一种模型优化方法，衍生和筛选的效果，一律以模型的最终结果来进行评定。而围绕集成学习进行海量特征衍生和筛选，也才是特征衍生和筛选技术能发挥巨大价值的主战场。

&emsp;&emsp;<font face="仿宋">而在抵达了单模的极限后，我们就会进入到下一阶段，也就是模型融合阶段。需要知道的是，只有单模的效果到达了极限，进一步的多模型融合、甚至多层融合，才是有意义的，才是有效果的。

# <center>Part 4.集成算法的训练与优化技巧

In [11]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin

# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

&emsp;&emsp;然后执行Part 1中的数据清洗相关工作：

In [12]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [13]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [14]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [15]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

## <center>Ch.1 海量特征衍生与筛选

- 海量特征的特征筛选

&emsp;&emsp;在特征筛选的最后一部分，我们开始讨论关于海量特征的特征筛选方法。正如上一小节讨论的一样，建模的目的不同，相应的特征工程策略也会有所不同。而在上一小节中，我们已经详细介绍了关于小量样本情况下的特征筛选方法，而如果是追求建模的最终预测效果，则不可避免会在特征衍生环节通过批量特征衍生来创造大量特征，而此时的特征筛选，也就必然是面对海量特征来进行筛选。

&emsp;&emsp;当然，除了面对的特征数量不同外，效果优先的机器学习建模策略也往往会采用集成学习+模型融合的策略，因此，我们筛选出来的特征也不仅仅是为了单独模型效果服务，而是为了最终融合结果服务。而这样的特征筛选过程，也必然要综合考虑模型融合的建模要求。因此，本节开始也将补充部分模型融合的基础知识，而更多的融合理论及策略，则将在下一部分详细介绍。

- 特征筛选整体流程

&emsp;&emsp;总的来说，模型效果优先的特征衍生和特征筛选并不是割裂的，甚至和模型训练也不是割裂的，而是相互关联、相互穿插的。也就是说我们完全可以按照特征衍生-特征筛选-模型训练-根据模型效果再次进行特征衍生-再次进行特征筛选-再次模型训练等步骤来执行，而在这个过程中，我们往往可以根据下游步骤的结果来调整上游操作，例如可以根据筛选结果调整衍生策略、可以根据模型训练结果调整筛选策略等。也就是说，实际的特征工程和模型训练，并没有完美的、固定的流程，要设计一个行之有效的特征工程和模型训练的策略，还是会比较考验建模工作者的实际经验。

- 特征筛选过程的核心矛盾

&emsp;&emsp;当然，从根据这个流程，我们也不难发现，特征筛选过程的核心矛盾仍然还是效果和效率的平衡，面对海量特征，我们无法做到非常精确的评估（例如是10000个特征最好还是10001个特征最好），当然，考虑到特征衍生环节也没有最优解，因此特征筛选过程寻求这个个证数量的最优解也是毫无意义的。面对最终要执行的模型融合，如何快速的筛选出一批相对高质量的特征，才是核心需要考虑的问题。

&emsp;&emsp;而接下来围绕本案例的数据集进行特征工程和模型训练，则是整个案例阶段的第一次尝试。我们将根据当前数据集的实际情况来设计特征衍生、筛选和模型训练的流程，并最终借助模型融合的方法，达到一个更好的建模效果。

## 一、基本思路

&emsp;&emsp;首先需要执行的就是特征衍生相关工作，考虑到本数据集样本数量有限，为了更好的做到效果和效率之间的平衡，此处考虑采用更为稳妥的特征衍生策略，即在特征衍生阶段更加侧重衍生特征的质量而非数量，这就需要在衍生的各个阶段配合进行同步的特征筛选相关工作。

- 原始特征的特征筛选

&emsp;&emsp;在实际执行特征衍生之前，我们需要围绕原始特征进行最基础的过滤——也就是缺失值过滤和方差过滤，需要剔除那些缺失值占比极高、或者方差为0的特征。不过根据此前数据探索的结果，原始变量中并不存在这类特征，因此本环节可以直接跳过。

> 需要注意的是，一般来说围绕原始特征的特征筛选工作都会一定程度放宽要求，除非是非常肯定的无用特征，否则一般都会考虑保留，以最大程度保留衍生特征的可能性原始数据集的完整信息（当然，原始数据集特征非常多的情况除外）。而如果是衍生特征，则可以根据实际情况提高筛选门槛，以提高后续模型筛选和训练的效率。

- 时序衍生特征

&emsp;&emsp;由于本数据时序特征较为特殊，在此前的数据准备过程中已经手动完成了相关特征的衍生工作：

In [16]:
X_train_seq.head()

,tenure_year_2014,tenure_year_2015,tenure_year_2016,tenure_year_2017,tenure_year_2018,tenure_year_2019,tenure_year_2020,tenure_month_1,tenure_month_2,tenure_month_3,...,tenure_month_7,tenure_month_8,tenure_month_9,tenure_month_10,tenure_month_11,tenure_month_12,tenure_quarter_1,tenure_quarter_2,tenure_quarter_3,tenure_quarter_4
4067,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3306,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3391,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3249,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2674,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
X_train_seq.shape

(5282, 23)

时序衍生均为离散特征，考虑到后续将纳入这些特征进行交叉分组，因此创建一个包含所有离散特征变量名称的对象cat_all：

In [18]:
cat_all = (category_cols + list(X_train_seq.columns)).copy()
cat_all

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'tenure_year_2014',
 'tenure_year_2015',
 'tenure_year_2016',
 'tenure_year_2017',
 'tenure_year_2018',
 'tenure_year_2019',
 'tenure_year_2020',
 'tenure_month_1',
 'tenure_month_2',
 'tenure_month_3',
 'tenure_month_4',
 'tenure_month_5',
 'tenure_month_6',
 'tenure_month_7',
 'tenure_month_8',
 'tenure_month_9',
 'tenure_month_10',
 'tenure_month_11',
 'tenure_month_12',
 'tenure_quarter_1',
 'tenure_quarter_2',
 'tenure_quarter_3',
 'tenure_quarter_4']

- 写入本地

&emsp;&emsp;为了方便后续单独调用某部分衍生特征，考虑将时序衍生特征写入本地：

In [19]:
X_train_seq.to_csv('featuresCreation/X_train_seq.csv', index=False)
X_test_seq.to_csv('featuresCreation/X_test_seq.csv', index=False)

## 二、交叉组合特征衍生与筛选

### 1.交叉组合特征衍生过程

&emsp;&emsp;然后来进行交叉组合特征衍生。由于交叉组合并不会衍生太多特征，因此可以考虑带入包括时序特征在内的全部离散特征，并进行两两组合：

In [20]:
Cross_Combination?

Signature: Cross_Combination(colNames, X_train, X_test, multi=False, OneHot=True)
Docstring:
交叉组合特征衍生函数

:param colNames: 参与交叉衍生的列名称
:param X_train: 训练集特征
:param X_test: 测试集特征
:param multi: 是否进行多变量交叉组合
:param OneHot: 是否进行独热编码

:return：交叉衍生后的新特征和特征名称
File:      d:\work\jupyter\telco\features_creation.py
Type:      function


In [21]:
# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

# 拼接数据集
train_temp = pd.concat([X_train, X_train_seq], axis=1)
test_temp = pd.concat([X_test, X_test_seq], axis=1)

# 带有时序特征的交叉组合      
CrossComb_train, CrossComb_test, colNames_train_new, colNames_test_new = Cross_Combination(cat_all, 
                                                                                           train_temp, 
                                                                                           test_temp)

In [22]:
CrossComb_train.head()

,gender&SeniorCitizen_Female&0,gender&SeniorCitizen_Female&1,gender&SeniorCitizen_Male&0,gender&SeniorCitizen_Male&1,gender&Partner_Female&No,gender&Partner_Female&Yes,gender&Partner_Male&No,gender&Partner_Male&Yes,gender&Dependents_Female&No,gender&Dependents_Female&Yes,...,tenure_quarter_1&tenure_quarter_4_1.0&0.0,tenure_quarter_2&tenure_quarter_3_0.0&0.0,tenure_quarter_2&tenure_quarter_3_0.0&1.0,tenure_quarter_2&tenure_quarter_3_1.0&0.0,tenure_quarter_2&tenure_quarter_4_0.0&0.0,tenure_quarter_2&tenure_quarter_4_0.0&1.0,tenure_quarter_2&tenure_quarter_4_1.0&0.0,tenure_quarter_3&tenure_quarter_4_0.0&0.0,tenure_quarter_3&tenure_quarter_4_0.0&1.0,tenure_quarter_3&tenure_quarter_4_1.0&0.0
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [23]:
CrossComb_train.shape

(5282, 3589)

离散特征在两两组合情况下总共衍生3589个特征。

### 2.交叉组合特征筛选过程

- 方差过滤

&emsp;&emsp;接下来，围绕交叉组合衍生特征进行特征筛选。这里首先可以考虑进行方差过滤。需要注意的是，交叉组合衍生的特征都是二分类离散变量，为了提高整体衍生特征质量，我们可以设置一个更高的方差阈值。这里考虑剔除少数类样本:多数类样本比例低于1:99的特征，即以0.01 * 0.99 = 0.0099为阈值，进行方差过滤：

In [24]:
0.01 * 0.99

0.0099

In [25]:
from sklearn.feature_selection import VarianceThreshold

In [26]:
sel = VarianceThreshold()
sel.fit(CrossComb_train)

VarianceThreshold()

In [27]:
CrossComb_cols = CrossComb_train.columns[sel.variances_ > 0.0099]
CrossComb_cols

Index(['gender&SeniorCitizen_Female&0', 'gender&SeniorCitizen_Female&1',
       'gender&SeniorCitizen_Male&0', 'gender&SeniorCitizen_Male&1',
       'gender&Partner_Female&No', 'gender&Partner_Female&Yes',
       'gender&Partner_Male&No', 'gender&Partner_Male&Yes',
       'gender&Dependents_Female&No', 'gender&Dependents_Female&Yes',
       ...
       'tenure_quarter_1&tenure_quarter_4_1.0&0.0',
       'tenure_quarter_2&tenure_quarter_3_0.0&0.0',
       'tenure_quarter_2&tenure_quarter_3_0.0&1.0',
       'tenure_quarter_2&tenure_quarter_3_1.0&0.0',
       'tenure_quarter_2&tenure_quarter_4_0.0&0.0',
       'tenure_quarter_2&tenure_quarter_4_0.0&1.0',
       'tenure_quarter_2&tenure_quarter_4_1.0&0.0',
       'tenure_quarter_3&tenure_quarter_4_0.0&0.0',
       'tenure_quarter_3&tenure_quarter_4_0.0&1.0',
       'tenure_quarter_3&tenure_quarter_4_1.0&0.0'],
      dtype='object', length=3474)

能够发现，此处剔除了100多个特征。

- 卡方检验

&emsp;&emsp;然后继续执行基于标签关联度的特征筛选。这里我们可以同时进行卡方检验和互信息法计算，然后选择两种方法挑选出来特征的交集（若希望选取更多衍生特征，也可以考虑二者的并集）。

&emsp;&emsp;首先先进行卡方检验，并以0.01为阈值进行特征筛选：

In [28]:
from sklearn.feature_selection import chi2

In [29]:
CrossComb_train[CrossComb_cols].head()

,gender&SeniorCitizen_Female&0,gender&SeniorCitizen_Female&1,gender&SeniorCitizen_Male&0,gender&SeniorCitizen_Male&1,gender&Partner_Female&No,gender&Partner_Female&Yes,gender&Partner_Male&No,gender&Partner_Male&Yes,gender&Dependents_Female&No,gender&Dependents_Female&Yes,...,tenure_quarter_1&tenure_quarter_4_1.0&0.0,tenure_quarter_2&tenure_quarter_3_0.0&0.0,tenure_quarter_2&tenure_quarter_3_0.0&1.0,tenure_quarter_2&tenure_quarter_3_1.0&0.0,tenure_quarter_2&tenure_quarter_4_0.0&0.0,tenure_quarter_2&tenure_quarter_4_0.0&1.0,tenure_quarter_2&tenure_quarter_4_1.0&0.0,tenure_quarter_3&tenure_quarter_4_0.0&0.0,tenure_quarter_3&tenure_quarter_4_0.0&1.0,tenure_quarter_3&tenure_quarter_4_1.0&0.0
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [30]:
chi2(CrossComb_train[CrossComb_cols], y_train)

(array([4.70486948e+00, 5.34073272e+01, 1.26267171e+01, ...,
        9.90547662e+01, 1.46860817e+02, 1.10886375e-03]),
 array([3.00772856e-02, 2.71083872e-13, 3.80272424e-04, ...,
        2.45614672e-23, 8.41700181e-34, 9.73435668e-01]))

In [31]:
chi2_p = chi2(CrossComb_train[CrossComb_cols], y_train)[1]

In [32]:
chi2_CrossComb_cols = []

for pValue, colname in zip(chi2_p, CrossComb_cols):
    if pValue < 0.01:
        chi2_CrossComb_cols.append(colname)

print(len(chi2_CrossComb_cols))

2495


最终挑选出2495个特征。

- 互信息法特征筛选

&emsp;&emsp;接下来，继续进行互信息法的特征筛选：

In [33]:
from sklearn.feature_selection import mutual_info_classif

In [34]:
MI = mutual_info_classif(CrossComb_train[CrossComb_cols], y_train, discrete_features=True, random_state=22)
MI

array([7.68870178e-04, 5.03288801e-03, 2.09448319e-03, ...,
       1.80071779e-02, 1.94739071e-02, 1.34373500e-07])

&emsp;&emsp;此处仍然可以采用众数量级后推小数点后两位的方法进行特征筛选，即设置0.1 * mean为阈值进行特征筛选：

In [35]:
MI.mean()

0.010353000716212672

In [36]:
MI_threshold = MI.mean() * 0.1
MI_threshold

0.0010353000716212671

In [37]:
MI_CrossComb_cols = []

for MIvalue, colname in zip(MI, CrossComb_cols):
    if MIvalue > MI_threshold:
        MI_CrossComb_cols.append(colname)

print(len(MI_CrossComb_cols))        

2419


然后取方差分析和互信息法挑选出来特征的交集，可以通过如下方法实现：

In [38]:
set(chi2_CrossComb_cols) & set(MI_CrossComb_cols)

{'tenure_month_2&tenure_month_12_1.0&0.0',
 'MultipleLines&tenure_year_2014_Yes&0.0',
 'gender&PaymentMethod_Female&Mailed check',
 'tenure_year_2019&tenure_month_6_1.0&0.0',
 'PhoneService&StreamingMovies_Yes&No',
 'StreamingTV&tenure_month_7_Yes&0.0',
 'Dependents&tenure_month_11_No&1.0',
 'Partner&OnlineSecurity_No&Yes',
 'InternetService&tenure_quarter_3_No&0.0',
 'OnlineSecurity&DeviceProtection_No internet service&No internet service',
 'PaymentMethod&tenure_year_2018_Credit card (automatic)&0.0',
 'DeviceProtection&PaymentMethod_Yes&Electronic check',
 'PaymentMethod&tenure_month_2_Electronic check&0.0',
 'TechSupport&Contract_No internet service&Month-to-month',
 'Dependents&tenure_year_2016_No&0.0',
 'MultipleLines&PaymentMethod_Yes&Electronic check',
 'tenure_year_2017&tenure_month_11_0.0&1.0',
 'PaymentMethod&tenure_month_7_Bank transfer (automatic)&0.0',
 'gender&tenure_quarter_4_Female&0.0',
 'SeniorCitizen&tenure_month_8_0&0.0',
 'PhoneService&tenure_quarter_4_Yes&0.0',
 

In [39]:
CrossComb_cols_select = list(set(chi2_CrossComb_cols) & set(MI_CrossComb_cols))
len(CrossComb_cols_select)

2369

从最终结果能够看出，卡方检验和互信息法筛选出来的特征还是高度一致的，而CrossComb_cols_select也就是最终筛选出来的交叉组合衍生特征。

> 这里需要注意，由于上述过程涉及集合对象的创建，而集合是无序对象，因此最终输出的CrossComb_cols_select特征顺序会发生改变。

- 写入本地

&emsp;&emsp;同样，我们将上述衍生特征写入本地：

In [40]:
CrossComb_train[CrossComb_cols_select]

,tenure_month_2&tenure_month_12_1.0&0.0,MultipleLines&tenure_year_2014_Yes&0.0,gender&PaymentMethod_Female&Mailed check,tenure_year_2019&tenure_month_6_1.0&0.0,PhoneService&StreamingMovies_Yes&No,StreamingTV&tenure_month_7_Yes&0.0,Dependents&tenure_month_11_No&1.0,Partner&OnlineSecurity_No&Yes,InternetService&tenure_quarter_3_No&0.0,OnlineSecurity&DeviceProtection_No internet service&No internet service,...,tenure_year_2017&tenure_month_5_0.0&1.0,tenure_month_12&tenure_quarter_4_0.0&0.0,tenure_year_2014&tenure_month_9_0.0&0.0,tenure_month_11&tenure_quarter_4_0.0&0.0,PaymentMethod&tenure_quarter_4_Mailed check&0.0,tenure_year_2015&tenure_month_12_0.0&1.0,Dependents&tenure_year_2020_Yes&0.0,tenure_year_2014&tenure_quarter_3_1.0&0.0,StreamingMovies&tenure_month_1_No&1.0,Partner&tenure_quarter_4_No&1.0
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5278,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5279,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [41]:
CrossComb_train[CrossComb_cols_select].to_csv('featuresCreation/X_train_CrossComb.csv', index=False)
CrossComb_test[CrossComb_cols_select].to_csv('featuresCreation/X_test_CrossComb.csv', index=False)

## 三、多项式特征衍生与筛选

### 1.多项式特征衍生过程

&emsp;&emsp;接下来进一步考虑多项式特征衍生。由于数据集只存在三个连续变量，可以考虑进行最高三阶多项式的、带入全部连续变量的两两组合多项式组合特征衍生：

In [42]:
Polynomial_Features?

Signature: Polynomial_Features(colNames, degree, X_train, X_test, multi=False)
Docstring:
多项式特征衍生函数

:param colNames: 参与交叉衍生的列名称
:param degree: 多项式最高阶
:param X_train: 训练集特征
:param X_test: 测试集特征
:param multi: 是否进行多变量多项式组衍生

:return：多项式衍生后的新特征和新列名称
File:      d:\work\jupyter\telco\features_creation.py
Type:      function


In [43]:
Poly_train, Poly_test, colNames_train_new, colNames_test_new = Polynomial_Features(numeric_cols, 
                                                                                   3, 
                                                                                   X_train, 
                                                                                   X_test, 
                                                                                   multi=False)

In [44]:
Poly_train.head()

,tenure**2*MonthlyCharges**0,tenure**1*MonthlyCharges**1,tenure**0*MonthlyCharges**2,tenure**3*MonthlyCharges**0,tenure**2*MonthlyCharges**1,tenure**1*MonthlyCharges**2,tenure**0*MonthlyCharges**3,tenure**2*TotalCharges**0,tenure**1*TotalCharges**1,tenure**0*TotalCharges**2,...,tenure**2*TotalCharges**1,tenure**1*TotalCharges**2,tenure**0*TotalCharges**3,MonthlyCharges**2*TotalCharges**0,MonthlyCharges**1*TotalCharges**1,MonthlyCharges**0*TotalCharges**2,MonthlyCharges**3*TotalCharges**0,MonthlyCharges**2*TotalCharges**1,MonthlyCharges**1*TotalCharges**2,MonthlyCharges**0*TotalCharges**3
0,4624.0,5412.8,6336.1600,314432.0,368070.4,430858.880,504358.336000,4624.0,375074.4,3.042405e+07,...,25505059.2,2.068835e+09,1.678130e+11,6336.1600,439057.6800,3.042405e+07,504358.336000,3.494899e+07,2.421754e+09,1.678130e+11
1,9.0,240.0,6400.0000,27.0,720.0,19200.000,512000.000000,9.0,723.9,5.822569e+04,...,2171.7,1.746771e+05,1.404986e+07,6400.0000,19304.0000,5.822569e+04,512000.000000,1.544320e+06,4.658055e+06,1.404986e+07
2,16.0,76.0,361.0000,64.0,304.0,1444.000,6859.000000,16.0,293.8,5.394903e+03,...,1175.2,2.157961e+04,3.962556e+05,361.0000,1395.5500,5.394903e+03,6859.000000,2.651545e+04,1.025031e+05,3.962556e+05
3,100.0,555.5,3085.8025,1000.0,5555.0,30858.025,171416.328875,100.0,5513.0,3.039317e+05,...,55130.0,3.039317e+06,1.675575e+08,3085.8025,30624.7150,3.039317e+05,171416.328875,1.701203e+06,1.688341e+07,1.675575e+08
4,16.0,80.2,402.0025,64.0,320.8,1608.010,8060.150125,16.0,365.8,8.363103e+03,...,1463.2,3.345241e+04,7.648057e+05,402.0025,1833.5725,8.363103e+03,8060.150125,3.676313e+04,1.676802e+05,7.648057e+05


In [45]:
Poly_train.shape

(5282, 21)

多项式特征衍生总共创建21个新特征。

### 2.多项式衍生特征的特征筛选

- 方差过滤

&emsp;&emsp;首先是方差过滤。多项式计算过程并不会造成大量零值，因此衍生特征中也不会存在方差为0的特征：

In [46]:
sel = VarianceThreshold()
sel.fit(Poly_train)

VarianceThreshold()

In [47]:
Poly_cols = Poly_train.columns[sel.variances_ > 0]
Poly_cols = list(Poly_cols)
Poly_cols

['tenure**2*MonthlyCharges**0',
 'tenure**1*MonthlyCharges**1',
 'tenure**0*MonthlyCharges**2',
 'tenure**3*MonthlyCharges**0',
 'tenure**2*MonthlyCharges**1',
 'tenure**1*MonthlyCharges**2',
 'tenure**0*MonthlyCharges**3',
 'tenure**2*TotalCharges**0',
 'tenure**1*TotalCharges**1',
 'tenure**0*TotalCharges**2',
 'tenure**3*TotalCharges**0',
 'tenure**2*TotalCharges**1',
 'tenure**1*TotalCharges**2',
 'tenure**0*TotalCharges**3',
 'MonthlyCharges**2*TotalCharges**0',
 'MonthlyCharges**1*TotalCharges**1',
 'MonthlyCharges**0*TotalCharges**2',
 'MonthlyCharges**3*TotalCharges**0',
 'MonthlyCharges**2*TotalCharges**1',
 'MonthlyCharges**1*TotalCharges**2',
 'MonthlyCharges**0*TotalCharges**3']

In [48]:
len(Poly_cols)

21

- 方差分析

&emsp;&emsp;接下来继续进行基于关联度指标的特征筛选。多项式衍生特征都是连续变量，可以考虑方差分析和互信息计算，然后挑选二者选出特征的并集作为最终特征筛选结果。

&emsp;&emsp;首先是方差分析特征筛选过程：

In [49]:
from sklearn.feature_selection import f_classif

In [50]:
f_classif(Poly_train, y_train)

(array([613.06960446, 245.40152491, 124.76344154, 515.91186261,
        292.16044487, 119.32037883,  78.31501911, 613.06960446,
        291.00481002, 172.88676712, 515.91186261, 282.94514445,
        181.83285997, 135.86888563, 124.76344154, 118.52840666,
        172.88676712,  78.31501911,  79.18553416, 125.32427404,
        135.86888563]),
 array([3.82932349e-128, 4.24721102e-054, 1.19847172e-028, 4.75460035e-109,
        8.52211388e-064, 1.75059872e-027, 1.17981151e-018, 3.82932349e-128,
        1.47627483e-063, 7.03410902e-039, 4.75460035e-109, 6.83637464e-062,
        9.06241025e-041, 5.09799183e-031, 1.19847172e-028, 2.58680629e-027,
        7.03410902e-039, 1.17981151e-018, 7.64280682e-019, 9.09338472e-029,
        5.09799183e-031]))

In [51]:
f_classif_p = f_classif(Poly_train, y_train)[1]

In [52]:
f_classif_Poly_cols = []

for pValue, colname in zip(f_classif_p, Poly_cols):
    if pValue < 0.01:
        f_classif_Poly_cols.append(colname)

print(len(f_classif_Poly_cols))

21


&emsp;&emsp;能够看出，多项式衍生的特征质量都比较高。

- 互信息法

&emsp;&emsp;接下来继续执行互信息法的计算，由于全部衍生特征都是连续变量，因此在调用mutual_info_classif计算时不用进行额外参数设置：

In [53]:
MI = mutual_info_classif(Poly_train[Poly_cols], y_train)
MI

array([0.08244379, 0.06146415, 0.04667433, 0.07714242, 0.07425349,
       0.0271758 , 0.04536536, 0.0791969 , 0.04709366, 0.04340931,
       0.07998568, 0.07682973, 0.0618343 , 0.04272974, 0.04856117,
       0.04318058, 0.04326713, 0.04812398, 0.0258615 , 0.03179083,
       0.04407073])

然后同样选取mean\*0.1作为阈值进行特征筛选：

In [54]:
MI.mean()

0.05383117009190194

In [55]:
MI_threshold = MI.mean() * 0.1
MI_threshold

0.005383117009190194

In [56]:
MI_Ploy_cols = []

for MIvalue, colname in zip(MI, Poly_cols):
    if MIvalue > MI_threshold:
        MI_Ploy_cols.append(colname)

print(len(MI_Ploy_cols))        

21


筛选结果仍然是保留全部21个衍生特征。因此，结合方差过滤、方差分析的结果，最终围绕多项式衍生的特征结果是保留全部特征：

In [57]:
Poly_cols_select = Poly_cols
Poly_cols_select

['tenure**2*MonthlyCharges**0',
 'tenure**1*MonthlyCharges**1',
 'tenure**0*MonthlyCharges**2',
 'tenure**3*MonthlyCharges**0',
 'tenure**2*MonthlyCharges**1',
 'tenure**1*MonthlyCharges**2',
 'tenure**0*MonthlyCharges**3',
 'tenure**2*TotalCharges**0',
 'tenure**1*TotalCharges**1',
 'tenure**0*TotalCharges**2',
 'tenure**3*TotalCharges**0',
 'tenure**2*TotalCharges**1',
 'tenure**1*TotalCharges**2',
 'tenure**0*TotalCharges**3',
 'MonthlyCharges**2*TotalCharges**0',
 'MonthlyCharges**1*TotalCharges**1',
 'MonthlyCharges**0*TotalCharges**2',
 'MonthlyCharges**3*TotalCharges**0',
 'MonthlyCharges**2*TotalCharges**1',
 'MonthlyCharges**1*TotalCharges**2',
 'MonthlyCharges**0*TotalCharges**3']

- 写入本地

&emsp;&emsp;同样，我们将上述衍生特征写入本地：

In [58]:
Poly_train[Poly_cols_select]

,tenure**2*MonthlyCharges**0,tenure**1*MonthlyCharges**1,tenure**0*MonthlyCharges**2,tenure**3*MonthlyCharges**0,tenure**2*MonthlyCharges**1,tenure**1*MonthlyCharges**2,tenure**0*MonthlyCharges**3,tenure**2*TotalCharges**0,tenure**1*TotalCharges**1,tenure**0*TotalCharges**2,...,tenure**2*TotalCharges**1,tenure**1*TotalCharges**2,tenure**0*TotalCharges**3,MonthlyCharges**2*TotalCharges**0,MonthlyCharges**1*TotalCharges**1,MonthlyCharges**0*TotalCharges**2,MonthlyCharges**3*TotalCharges**0,MonthlyCharges**2*TotalCharges**1,MonthlyCharges**1*TotalCharges**2,MonthlyCharges**0*TotalCharges**3
0,4624.0,5412.80,6336.1600,314432.0,368070.40,430858.8800,5.043583e+05,4624.0,375074.40,3.042405e+07,...,25505059.20,2.068835e+09,1.678130e+11,6336.1600,439057.6800,3.042405e+07,5.043583e+05,3.494899e+07,2.421754e+09,1.678130e+11
1,9.0,240.00,6400.0000,27.0,720.00,19200.0000,5.120000e+05,9.0,723.90,5.822569e+04,...,2171.70,1.746771e+05,1.404986e+07,6400.0000,19304.0000,5.822569e+04,5.120000e+05,1.544320e+06,4.658055e+06,1.404986e+07
2,16.0,76.00,361.0000,64.0,304.00,1444.0000,6.859000e+03,16.0,293.80,5.394903e+03,...,1175.20,2.157961e+04,3.962556e+05,361.0000,1395.5500,5.394903e+03,6.859000e+03,2.651545e+04,1.025031e+05,3.962556e+05
3,100.0,555.50,3085.8025,1000.0,5555.00,30858.0250,1.714163e+05,100.0,5513.00,3.039317e+05,...,55130.00,3.039317e+06,1.675575e+08,3085.8025,30624.7150,3.039317e+05,1.714163e+05,1.701203e+06,1.688341e+07,1.675575e+08
4,16.0,80.20,402.0025,64.0,320.80,1608.0100,8.060150e+03,16.0,365.80,8.363103e+03,...,1463.20,3.345241e+04,7.648057e+05,402.0025,1833.5725,8.363103e+03,8.060150e+03,3.676313e+04,1.676802e+05,7.648057e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,2704.0,5527.60,11299.6900,140608.0,287435.20,587583.8800,1.201157e+06,2704.0,285324.00,3.010717e+07,...,14836848.00,1.565573e+09,1.651980e+11,11299.6900,583268.1000,3.010717e+07,1.201157e+06,6.200140e+07,3.200392e+09,1.651980e+11
5278,256.0,865.60,2926.8100,4096.0,13849.60,46828.9600,1.583404e+05,256.0,14224.00,7.903210e+05,...,227584.00,1.264514e+07,7.025954e+08,2926.8100,48094.9000,7.903210e+05,1.583404e+05,2.601934e+06,4.275637e+07,7.025954e+08
5279,784.0,2972.20,11267.8225,21952.0,83221.60,315499.0300,1.196079e+06,784.0,88270.00,9.938256e+06,...,2471560.00,2.782712e+08,3.133035e+10,11267.8225,334637.8750,9.938256e+06,1.196079e+06,3.552181e+07,1.054946e+09,3.133035e+10
5280,225.0,305.25,414.1225,3375.0,4578.75,6211.8375,8.427393e+03,225.0,5039.25,1.128624e+05,...,75588.75,1.692936e+06,3.791612e+07,414.1225,6836.5825,1.128624e+05,8.427393e+03,1.391245e+05,2.296750e+06,3.791612e+07


In [59]:
Poly_train[Poly_cols_select].to_csv('featuresCreation/X_train_Poly.csv', index=False)
Poly_test[Poly_cols_select].to_csv('featuresCreation/X_test_Poly.csv', index=False)

## 四、分组统计特征衍生与筛选

### 1.分组统计特征衍生

&emsp;&emsp;接下来进行分组统计特征衍生，此处若如果带入全部离散特征进行带拓展项的分组统计特征衍生，则会创建近10万条特征。考虑到数据集本身样本数量只有五千多条，我们可以考虑借助此前探索得到的“分组衍生特征性能和KeyCol原始性能接近”的规律，提前对KeyCol进行筛选，只挑选那些表现较好的KeyCol进行分组统计特征衍生，以提高整体执行效率。

- KeyCol筛选

&emsp;&emsp;这里围绕cat_all离散变量的筛选可以用卡方检验也可以用互信息法，这里我们采用一种组合方法，即先用卡方检验剔除显著性不到0.01的特征，再用互信息法剔除那些MI值明显小于众数的特征，然后最终取交集。具体执行流程如下：

In [60]:
# 拼接数据集
train_temp_OE = pd.concat([X_train_OE, X_train_seq], axis=1)
test_temp_OE = pd.concat([X_test_OE, X_test_seq], axis=1)

In [61]:
train_temp_OE[cat_all]

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,...,tenure_month_7,tenure_month_8,tenure_month_9,tenure_month_10,tenure_month_11,tenure_month_12,tenure_quarter_1,tenure_quarter_2,tenure_quarter_3,tenure_quarter_4
4067,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3306,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3391,1.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3249,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2674,0.0,1.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,1.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,2.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
356,0.0,1.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4908,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,2.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6276,0.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [62]:
y_train

4067    0
3306    0
3391    0
3249    0
2674    0
       ..
5478    0
356     0
4908    1
6276    0
2933    0
Name: Churn, Length: 5282, dtype: int64

In [63]:
chi2(train_temp_OE[cat_all], y_train)

(array([9.11414942e-01, 8.73653155e+01, 6.99267855e+01, 8.58416151e+01,
        3.28337137e-03, 5.05565036e+00, 9.87028913e+00, 4.70600801e+02,
        1.79111384e+02, 1.66587107e+02, 4.13243802e+02, 6.73755161e+00,
        1.06279915e+01, 8.30862655e+02, 7.51550195e+01, 5.15568690e+01,
        2.18414361e+02, 4.71022384e+01, 2.04721265e+01, 6.82195893e+00,
        2.93245371e+00, 3.73933423e+02, 3.54011792e+00, 8.08923275e+01,
        3.28456745e+01, 1.81976499e+00, 6.08334094e+00, 1.10199338e+01,
        1.68886049e+00, 1.83342748e+00, 3.25740687e-02, 1.42652376e+00,
        6.54213677e+00, 1.15481920e+01, 1.74805428e+02, 9.35057147e+01,
        1.66365418e+01, 1.10886375e-03, 1.46860817e+02]),
 array([3.39739272e-001, 9.02188129e-021, 6.15469914e-017, 1.94940242e-020,
        9.54305655e-001, 2.45457682e-002, 1.67969111e-003, 2.37153824e-104,
        7.57606574e-041, 4.11736822e-038, 7.21175221e-092, 9.44041024e-003,
        1.11388631e-003, 1.05225513e-182, 4.35169013e-018, 6.95529

In [64]:
chi2_p = chi2(train_temp_OE[cat_all], y_train)[1]

In [65]:
chi2_select_cols = []

for pValue, colname in zip(chi2_p, cat_all):
    if pValue < 0.01:
        chi2_select_cols.append(colname)

print(len(chi2_select_cols))
chi2_select_cols

26


['SeniorCitizen',
 'Partner',
 'Dependents',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'tenure_year_2014',
 'tenure_year_2015',
 'tenure_year_2016',
 'tenure_year_2017',
 'tenure_year_2019',
 'tenure_month_1',
 'tenure_month_2',
 'tenure_month_5',
 'tenure_month_11',
 'tenure_month_12',
 'tenure_quarter_1',
 'tenure_quarter_2',
 'tenure_quarter_4']

&emsp;&emsp;接下来继续执行互信息法特征筛选：

In [66]:
MI = mutual_info_classif(train_temp_OE[cat_all], y_train, discrete_features=True, random_state=22)
MI

array([1.73090078e-04, 9.13473736e-03, 1.29159500e-02, 1.24089586e-02,
       3.11949909e-06, 4.99279481e-04, 5.43286385e-02, 7.08580232e-02,
       4.78061879e-02, 4.70107520e-02, 6.57416875e-02, 3.30813850e-02,
       3.34860937e-02, 9.78731272e-02, 1.81578691e-02, 4.59939447e-02,
       3.20973276e-02, 5.65419301e-03, 2.33501821e-03, 7.58878530e-04,
       3.18699916e-04, 4.89210895e-02, 5.74417822e-04, 1.02552592e-02,
       3.81369383e-03, 1.90633022e-04, 6.48159371e-04, 1.19899259e-03,
       1.76110815e-04, 1.91702192e-04, 3.30476176e-06, 1.43599947e-04,
       6.48822228e-04, 1.14373671e-03, 1.75928217e-02, 1.30391564e-02,
       2.05498995e-03, 1.34373500e-07, 1.94739071e-02])

此处仍然可以采用均值*0.1作为阈值进行特征筛选：

In [67]:
MI_select_cols = []
MI_threshold = MI.mean() * 0.1

for MIvalue, colname in zip(MI, cat_all):
    if MIvalue > MI_threshold:
        MI_select_cols.append(colname)

print(len(MI_select_cols))        
MI_select_cols

23


['SeniorCitizen',
 'Partner',
 'Dependents',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'tenure_year_2014',
 'tenure_year_2015',
 'tenure_year_2016',
 'tenure_year_2019',
 'tenure_month_1',
 'tenure_month_2',
 'tenure_month_12',
 'tenure_quarter_1',
 'tenure_quarter_2',
 'tenure_quarter_4']

然后取方差分析和互信息法挑选出来特征的交集：

In [68]:
set(chi2_select_cols) & set(MI_select_cols)

{'Contract',
 'Dependents',
 'DeviceProtection',
 'InternetService',
 'OnlineBackup',
 'OnlineSecurity',
 'PaperlessBilling',
 'Partner',
 'PaymentMethod',
 'SeniorCitizen',
 'StreamingMovies',
 'StreamingTV',
 'TechSupport',
 'tenure_month_1',
 'tenure_month_12',
 'tenure_month_2',
 'tenure_quarter_1',
 'tenure_quarter_2',
 'tenure_quarter_4',
 'tenure_year_2014',
 'tenure_year_2015',
 'tenure_year_2016',
 'tenure_year_2019'}

接下来我们采用这些特征作为KeyCol来进行分组统计特征衍生：

In [69]:
keycol = list(set(chi2_select_cols) & set(MI_select_cols))

In [70]:
# 创建一个未被选中离散变量的list
cat_rest = []

for col in cat_all:
    if col not in keycol:
        cat_rest.append(col)

cat_rest

['gender',
 'PhoneService',
 'MultipleLines',
 'tenure_year_2017',
 'tenure_year_2018',
 'tenure_year_2020',
 'tenure_month_3',
 'tenure_month_4',
 'tenure_month_5',
 'tenure_month_6',
 'tenure_month_7',
 'tenure_month_8',
 'tenure_month_9',
 'tenure_month_10',
 'tenure_month_11',
 'tenure_quarter_3']

- 分组统计特征衍生

In [71]:
Group_Statistics?

Signature:
Group_Statistics(
    keyCol,
    X_train,
    X_test,
    col_num=None,
    col_cat=None,
    num_stat=['mean', 'var', 'max', 'min', 'skew', 'median'],
    cat_stat=['mean', 'var', 'max', 'min', 'median', 'count', 'nunique'],
    quant=True,
    multi=False,
    extension=False,
)
Docstring:
分组统计特征衍生函数

:param keyCol: 分组参考的关键变量
:param X_train: 训练集特征
:param X_test: 测试集特征
:param col_num: 参与衍生的连续型变量
:param col_cat: 参与衍生的离散型变量
:param num_stat: 连续变量分组统计量
:param cat_num: 离散变量分组统计量  
:param quant: 是否计算分位数  
:param multi: 是否进行多变量的分组统计特征衍生
:param extension: 是否进行二阶特征衍生

:return：分组统计衍生后的新特征和新特征的名称
File:      d:\work\jupyter\telco\features_creation.py
Type:      function


In [72]:
# 创建容器
col_temp = keycol.copy()
GroupStat_train = pd.DataFrame()
GroupStat_test = pd.DataFrame()

for i in range(len(col_temp)):
    keyCol = col_temp.pop(i)
    features_train1, features_test1, colNames_train, colNames_test = Group_Statistics(keyCol,
                                                                                      train_temp_OE,
                                                                                      test_temp_OE,
                                                                                      col_num=numeric_cols,
                                                                                      col_cat=col_temp+cat_rest, 
                                                                                      extension=True)
    
    GroupStat_train = pd.concat([GroupStat_train, features_train1],axis=1)
    GroupStat_test = pd.concat([GroupStat_test, features_test1],axis=1)
    
    col_temp = keycol.copy()

In [73]:
GroupStat_train.head()

,tenure_tenure_year_2015_mean,tenure_tenure_year_2015_var,tenure_tenure_year_2015_max,tenure_tenure_year_2015_min,tenure_tenure_year_2015_skew,tenure_tenure_year_2015_median,MonthlyCharges_tenure_year_2015_mean,MonthlyCharges_tenure_year_2015_var,MonthlyCharges_tenure_year_2015_max,MonthlyCharges_tenure_year_2015_min,...,tenure_month_3_cv_StreamingMovies,tenure_month_4_cv_StreamingMovies,tenure_month_5_cv_StreamingMovies,tenure_month_6_cv_StreamingMovies,tenure_month_7_cv_StreamingMovies,tenure_month_8_cv_StreamingMovies,tenure_month_9_cv_StreamingMovies,tenure_month_10_cv_StreamingMovies,tenure_month_11_cv_StreamingMovies,tenure_quarter_3_cv_StreamingMovies
0,29.409878,613.421998,72,0,0.523153,23.0,63.829731,889.033318,118.75,18.25,...,3.261447,3.484000,3.559100,3.723440,3.459903,3.625138,3.484000,3.738084,3.484000,1.861802
1,29.409878,613.421998,72,0,0.523153,23.0,63.829731,889.033318,118.75,18.25,...,3.852995,3.915736,3.915736,3.433904,3.601332,3.601332,3.295888,3.118448,3.002764,1.843026
2,29.409878,613.421998,72,0,0.523153,23.0,63.829731,889.033318,118.75,18.25,...,3.518062,4.068883,3.636373,3.939975,3.821737,3.793673,3.793673,3.388973,3.348697,2.038295
3,29.409878,613.421998,72,0,0.523153,23.0,63.829731,889.033318,118.75,18.25,...,3.852995,3.915736,3.915736,3.433904,3.601332,3.601332,3.295888,3.118448,3.002764,1.843026
4,29.409878,613.421998,72,0,0.523153,23.0,63.829731,889.033318,118.75,18.25,...,3.518062,4.068883,3.636373,3.939975,3.821737,3.793673,3.793673,3.388973,3.348697,2.038295


In [74]:
GroupStat_train.shape

(5282, 16905)

分组统计特征衍生最终创建了16905个特征。

### 2.分组统计衍生特征的特征筛选

- 分组统计特征变量类型讨论

&emsp;&emsp;接下来，考虑围绕分组统计衍生特征进行特征筛选。这里需要注意，分组统计衍生特征从原理层面来看应该属于离散变量，这些特征取值大小本身不仅具有标记作用，而且具有数值绝对大小意义（都是统计量的计算结果）。但同时，这些特征的数值分布和KeyCol一致，也就是尽管是连续变量，但取值个数有限。例如衍生特征'tenure_month_10_DeviceProtection_mean'，是tenure_month_10在DeviceProtection分组下组内均值计算结果，均值本身数值有数值大小意义，数值越小代表组内用户10月入网占比越少，但同时tenure_month_10_DeviceProtection_mean只有三个不同的取值，且分布和DeviceProtection数值分布相同：

In [60]:
GroupStat_train['tenure_month_10_DeviceProtection_mean']

0       0.088851
1       0.088851
2       0.080142
3       0.069307
4       0.080142
          ...   
5277    0.069307
5278    0.088851
5279    0.069307
5280    0.080142
5281    0.080142
Name: tenure_month_10_DeviceProtection_mean, Length: 5282, dtype: float64

In [61]:
GroupStat_train['tenure_month_10_DeviceProtection_mean'].nunique()

3

In [62]:
X_train_OE['DeviceProtection']

4067    0.0
3306    0.0
3391    1.0
3249    2.0
2674    1.0
       ... 
5478    2.0
356     0.0
4908    2.0
6276    1.0
2933    1.0
Name: DeviceProtection, Length: 5282, dtype: float64

In [63]:
X_train_OE['DeviceProtection'].nunique()

3

同时我们也能够观察到，很多时候由于被分组统计的都是0-1离散变量，因此不同组的部分统计量（如均值）的统计结果也都是在0-1之间，并且彼此差异并不大。

- 方差过滤

&emsp;&emsp;基于分组统计衍生特征的基本情况，我们可以制定如下特征筛选策略：首先，无需进行缺失值过滤，由于我们自定义的特征衍生函数并不会创造缺失值，而原始数据集中的缺失值已经完成了填补，因此无需进行缺失值过滤；其次，对于方差过滤来说，零方差过滤是必须要执行的，但小方差过滤意义不大，即只剔除那些零方差的特征，而不剔除小方差的特征。这么做的主要原因是很多统计结果数值都较小，并且同一列的不同取值差异并不大，如上述tenure_month_10_DeviceProtection_mean，而这些特征数值较小的根本原因因为被统计的离散变量数值较小，即tenure_month_10_DeviceProtection_mean特征取值较小的原因是tenure_month_10是0-1变量，而如果把0、1这两个数值标记改为0、100（tenure_month_10是名义变量，数值没有大小意义，可以任意修改数值标记），则衍生特征tenure_month_10_DeviceProtection_mean取值也会变大。但这种数值上的变化没有任何意义，因此从这个角度来说，无需对分组统计特征进行小方差特征过滤。

&emsp;&emsp;接下来对分组统计衍生特征进行0方差过滤：

In [75]:
sel = VarianceThreshold()
sel.fit(GroupStat_train)

VarianceThreshold()

In [76]:
GroupStat_cols = list(GroupStat_train.columns[sel.variances_ > 0])
len(GroupStat_cols)

11345

能够看出，0方差过滤还是剔除了很多特征的，最终剩下11345个特征。这里我们可以进一步观察0方差过滤到底剔除了那些特征：

In [66]:
set(GroupStat_train.columns) - set(GroupStat_cols)

{'PhoneService_SeniorCitizen_min',
 'tenure_quarter_2_Contract_max',
 'tenure_month_1_OnlineSecurity_min',
 'tenure_quarter_2_tenure_month_2_median',
 'PhoneService_tenure_year_2015_max',
 'tenure_month_8_mag2_tenure_quarter_2',
 'tenure_month_5_OnlineBackup_max',
 'Partner_StreamingTV_min',
 'StreamingTV_tenure_month_12_nunique',
 'PhoneService_tenure_month_1_q1',
 'tenure_month_2_mag2_tenure_year_2019',
 'tenure_month_5_DeviceProtection_median',
 'tenure_quarter_3_tenure_quarter_1_median',
 'InternetService_tenure_quarter_1_min',
 'gender_InternetService_nunique',
 'Contract_tenure_quarter_2_min',
 'Partner_TechSupport_q1',
 'tenure_year_2016_InternetService_median',
 'gender_gap_PaperlessBilling',
 'tenure_year_2019_TechSupport_median',
 'Dependents_tenure_month_2_q2',
 'tenure_month_12_tenure_year_2016_q1',
 'tenure_quarter_2_mag2_tenure_month_2',
 'TechSupport_tenure_quarter_1_q2',
 'tenure_month_9_Partner_nunique',
 'tenure_year_2017_mag2_tenure_month_12',
 'tenure_month_12_mag2_

In [67]:
GroupStat_train['tenure_quarter_4_PaperlessBilling_median']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5277    0.0
5278    0.0
5279    0.0
5280    0.0
5281    0.0
Name: tenure_quarter_4_PaperlessBilling_median, Length: 5282, dtype: float64

In [68]:
GroupStat_train['InternetService_PaymentMethod_max']

0       2.0
1       2.0
2       2.0
3       2.0
4       2.0
       ... 
5277    2.0
5278    2.0
5279    2.0
5280    2.0
5281    2.0
Name: InternetService_PaymentMethod_max, Length: 5282, dtype: float64

能够看出，大多数剔除的特征都是统计变量为离散变量时进行的数值分布规律的统计量，例如极值、四分位数等。仔细思考其实也不难理解，很多时候不同组内的离散变量取值分布较为类似，例如不同支付方式的用户都有没有购买互联网服务的情况，此时就会出现'InternetService_PaymentMethod_max'全都取值为2的情况。因此围绕分组统计衍生特征进行0方差的过滤是非常有必要的。

> 但这里需要注意，尽管很多离散变量不同组的极值、分位数的分组统计结果相同，但这并不代表极值、分位数等统计量不重要。恰好相反的是，正是因为很多离散变量的不同组的极值、分位数的分组统计结果相同，那些不同组内极值、分位数的分组统计结果不同的特征，往往是提升模型效果的关键。

而GroupStat_cols就是经过方差过滤后的特征名称列表。

- 方差分析

&emsp;&emsp;接下来继续考虑进行标签关联度指标特征筛选。正如此前所说，分组统计的特征本质上其实是连续变量，因此可以考虑方差分析与互信息法特征筛选。并且根据之前介绍的方差分析与互信息的基本原理可知，两种方法并不会受连续变量的绝对数值大小影响，因此可信度较高。

&emsp;&emsp;首先是方差分析，我们带入全部方差过滤后的GroupStat_cols进行检验：

In [77]:
f_classif_p = f_classif(GroupStat_train[GroupStat_cols], y_train)[1]

In [78]:
f_classif_GroupStat_cols = []

for pValue, colname in zip(f_classif_p, GroupStat_cols):
    if pValue < 0.01:
        f_classif_GroupStat_cols.append(colname)

print(len(f_classif_GroupStat_cols))

11123


- 互信息法

&emsp;&emsp;接下来继续执行互信息法的计算，由于全部衍生特征都是连续变量，因此在调用mutual_info_classif计算时不用进行额外参数设置：

In [79]:
MI = mutual_info_classif(GroupStat_train[GroupStat_cols], y_train, random_state=22)
MI

array([0.00454814, 0.        , 0.0120663 , ..., 0.03797375, 0.02559527,
       0.02455003])

然后同样选取mean\*0.1作为阈值进行特征筛选：

In [80]:
MI.mean()

0.03027058891310412

In [81]:
MI_threshold = MI.mean() * 0.1
MI_threshold

0.003027058891310412

In [82]:
MI_GroupStat_cols = []

for MIvalue, colname in zip(MI, GroupStat_cols):
    if MIvalue > MI_threshold:
        MI_GroupStat_cols.append(colname)

print(len(MI_GroupStat_cols))        

10210


最后将两种方法挑选出来的特征取交集，并得到最终筛选出来的特征GroupStat_cols_select：

In [83]:
GroupStat_cols_select = list(set(f_classif_GroupStat_cols) & set(MI_GroupStat_cols))
len(GroupStat_cols_select)

9988

总共有9988个特征，在整个初筛阶段，总共剔除了40%个衍生特征：

In [84]:
1 - 9988/16905

0.4091688849452825

> 需要注意的是，尽管我们在互信息法中设置了随机数种子，但由于本身数值精度较高，因此会在小数点后6位左右存在误差，这也会导致筛选结果会存在随机性。

- 写入本地

&emsp;&emsp;同样，我们将上述衍生特征写入本地：

In [85]:
GroupStat_train[GroupStat_cols_select]

,tenure_year_2016_norm_TechSupport,MultipleLines_tenure_year_2019_mean,InternetService_tenure_year_2019_mean,PaperlessBilling_InternetService_median,tenure_year_2014_PaymentMethod_q2,StreamingMovies_minus2_StreamingMovies_OnlineBackup_mean,Contract_minus2_Contract_OnlineBackup_mean,PaperlessBilling_minus1_PaperlessBilling_StreamingMovies_mean,tenure_month_6_minus2_tenure_month_6_DeviceProtection_mean,StreamingTV_minus1_StreamingTV_Contract_mean,...,Dependents_norm_tenure_year_2019,tenure_month_8_dive1_tenure_month_8_OnlineSecurity_mean,tenure_month_2_PaymentMethod_var,tenure_quarter_4_tenure_month_12_nunique,tenure_month_8_OnlineBackup_var,SeniorCitizen_mag1_Partner,tenure_month_5_minus1_tenure_month_5_tenure_month_2_mean,TotalCharges_Contract_q1,Contract_dive1_Contract_Dependents_mean,tenure_month_9_mag1_InternetService
0,5.776420,1.104424,0.862325,1.0,1.0,0.853945,1.130386,1.281925,-0.078599,0.731591,...,-0.725437,31.640827,0.080519,2,0.067971,-0.161278,-0.073541,1305.400,0.000000,-0.079502
1,5.776420,0.551430,0.863055,1.0,0.0,0.853945,1.130386,-0.625530,-0.078599,-0.794933,...,1.964283,0.000000,0.070756,2,0.067971,-0.161278,-0.073541,155.900,1.017231,-0.080952
2,2.786759,0.551430,0.863055,0.0,0.0,0.000000,-0.065895,0.701692,0.939448,-0.794933,...,1.964283,15.381195,0.056315,2,0.060833,-0.155135,-0.073541,155.900,0.000000,-0.065004
3,-0.331064,0.551430,0.863055,1.0,0.0,0.853945,1.130386,-0.625530,1.934543,-0.794933,...,1.964283,0.000000,0.056315,2,0.067971,-0.161278,-0.073541,155.900,1.017231,-0.079502
4,2.786759,0.551430,0.863055,0.0,0.0,0.000000,-0.065895,0.701692,0.939448,-0.794933,...,1.964283,15.381195,0.056315,2,0.060833,-0.155135,-0.073541,155.900,0.000000,-0.065004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,5.776420,1.104424,0.862325,1.0,0.0,0.853945,1.130386,1.281925,1.934543,0.731591,...,-0.725437,31.640827,0.070756,2,0.067971,-0.161278,-0.073541,1305.400,0.000000,-0.080952
5278,-0.331064,1.104424,0.862325,1.0,0.0,-0.853288,-0.368044,-0.625530,-0.078599,-0.794933,...,-0.725437,31.640827,0.070756,2,0.064750,-0.155135,-0.073541,155.900,0.000000,-0.079502
5279,-0.331064,1.104424,0.862325,1.0,0.0,0.853945,1.130386,1.281925,1.934543,-0.794933,...,-0.725437,0.000000,0.070756,2,0.067971,-0.155135,-0.073541,155.900,0.000000,-0.080952
5280,2.786759,1.104424,0.862325,0.0,1.0,0.000000,-0.065895,0.701692,0.939448,-0.146630,...,-0.725437,15.381195,0.082785,2,0.060833,-0.161278,-0.073541,955.975,1.017231,-0.065004


In [86]:
GroupStat_train[GroupStat_cols_select].to_csv('featuresCreation/X_train_GroupStat.csv', index=False)
GroupStat_test[GroupStat_cols_select].to_csv('featuresCreation/X_test_GroupStat.csv', index=False)

## 五、目标编码特征衍生与筛选

### 1.目标编码特征衍生过程

&emsp;&emsp;接下来继续进行目标编码，由于目标编码本身的计算特性，较强的KeyCol不一定能衍生出较强的特征，因此需要带入全部离散特征进行计算。此外，由于目标编码的特殊性，有效特征产出率较低，因此建议采用extension方式进行特征衍生，创建更多特征：

In [77]:
Target_Encode?

Signature:
Target_Encode(
    keyCol,
    X_train,
    y_train,
    X_test,
    col_num=None,
    col_cat=None,
    num_stat=['mean', 'var', 'max', 'min', 'skew', 'median'],
    cat_stat=['mean', 'var', 'max', 'min', 'median', 'count', 'nunique'],
    quant=True,
    multi=False,
    extension=False,
    n_splits=5,
    random_state=42,
)
Docstring:
目标编码

:param keyCol: 分组参考的关键变量
:param X_train: 训练集特征
:param y_train: 训练集标签
:param X_test: 测试集特征
:param col_num: 参与衍生的连续型变量
:param col_cat: 参与衍生的离散型变量
:param num_stat: 连续变量分组统计量
:param cat_num: 离散变量分组统计量  
:param quant: 是否计算分位数  
:param multi: 是否进行多变量的分组统计特征衍生
:param extension: 是否进行二阶特征衍生
:param n_splits: 进行几折交叉统计  
:param random_state: 随机数种子  

:return：目标编码后的新特征和新特征的名称
File:      d:\work\jupyter\telco\features_creation.py
Type:      function


In [87]:
# 定义标签
col_cat = [target]
print(col_cat)

# 创建容器
col_temp = cat_all.copy()
TarEnc_train = pd.DataFrame()
TarEnc_test = pd.DataFrame()

for keyCol in col_temp:
    features_train1, features_test1, colNames_train_new, colNames_test_new = Target_Encode(keyCol, 
                                                                                           train_temp_OE, 
                                                                                           y_train,
                                                                                           test_temp_OE, 
                                                                                           col_cat=col_cat, 
                                                                                           extension=True)
    
    TarEnc_train = pd.concat([TarEnc_train, features_train1],axis=1)
    TarEnc_test = pd.concat([TarEnc_test, features_test1],axis=1)
    
    col_temp = cat_all.copy()

['Churn']


In [88]:
TarEnc_train.head()

,Churn_gender_mean_kfold,Churn_gender_var_kfold,Churn_gender_max_kfold,Churn_gender_min_kfold,Churn_gender_median_kfold,Churn_gender_count_kfold,Churn_gender_nunique_kfold,Churn_gender_q1_kfold,Churn_gender_q2_kfold,Churn_dive1_Churn_gender_mean_kfold,...,Churn_tenure_quarter_4_q2_kfold,Churn_dive1_Churn_tenure_quarter_4_mean_kfold,Churn_dive2_Churn_tenure_quarter_4_median_kfold,Churn_minus1_Churn_tenure_quarter_4_mean_kfold,Churn_minus2_Churn_tenure_quarter_4_mean_kfold,Churn_norm_tenure_quarter_4_kfold,Churn_gap_tenure_quarter_4_kfold,Churn_mag1_tenure_quarter_4_kfold,Churn_mag2_tenure_quarter_4_kfold,Churn_cv_tenure_quarter_4_kfold
0,0.272425,0.198304,1.0,0.0,0.0,2107.0,2.0,0.0,1.0,0.000000,...,1.0,2.534907,100000.0,0.605518,0.605518,1.238452,1.0,-0.394482,0.0,1.239371
1,0.258141,0.191594,1.0,0.0,0.0,2119.0,2.0,0.0,1.0,3.873707,...,0.0,0.000000,0.0,-0.205199,-0.205199,-0.508011,0.0,-0.205199,0.0,1.968318
3,0.248839,0.187005,1.0,0.0,0.0,2154.0,2.0,0.0,0.0,4.018495,...,0.0,0.000000,0.0,-0.200000,-0.200000,-0.499901,0.0,-0.200000,0.0,2.000246
4,0.272425,0.198304,1.0,0.0,0.0,2107.0,2.0,0.0,1.0,0.000000,...,1.0,2.534907,100000.0,0.605518,0.605518,1.238452,1.0,-0.394482,0.0,1.239371
6,0.260540,0.192750,1.0,0.0,0.0,2111.0,2.0,0.0,1.0,3.838035,...,0.0,0.000000,0.0,-0.204026,-0.204026,-0.506184,0.0,-0.204026,0.0,1.975421


In [89]:
TarEnc_train.shape

(5282, 702)

最终目标编码创建了702个新特征。

### 2.目标编码特征筛选

&emsp;&emsp;接下来继续进行目标编码的衍生特征的特征筛选。目标编码也是某种意义上的分组统计，因此首先需要进行零值方差过滤，然后再考虑使用标签关联度指标进行特征筛选。

- 方差过滤

&emsp;&emsp;首先是方差过滤，这里仍然采用0值方差过滤：

In [90]:
sel = VarianceThreshold()
sel.fit(TarEnc_train)

VarianceThreshold()

In [91]:
TarEnc_cols = list(TarEnc_train.columns[sel.variances_ > 0])
len(TarEnc_cols)

456

能够看出，在0方差过滤中被剔除的特征也大都是组内分布统计量：

In [83]:
set(TarEnc_train.columns) - set(TarEnc_cols)

{'Churn_Contract_max_kfold',
 'Churn_Contract_median_kfold',
 'Churn_Contract_min_kfold',
 'Churn_Contract_nunique_kfold',
 'Churn_Contract_q1_kfold',
 'Churn_Dependents_max_kfold',
 'Churn_Dependents_median_kfold',
 'Churn_Dependents_min_kfold',
 'Churn_Dependents_nunique_kfold',
 'Churn_Dependents_q1_kfold',
 'Churn_DeviceProtection_max_kfold',
 'Churn_DeviceProtection_median_kfold',
 'Churn_DeviceProtection_min_kfold',
 'Churn_DeviceProtection_nunique_kfold',
 'Churn_DeviceProtection_q1_kfold',
 'Churn_InternetService_max_kfold',
 'Churn_InternetService_median_kfold',
 'Churn_InternetService_min_kfold',
 'Churn_InternetService_nunique_kfold',
 'Churn_InternetService_q1_kfold',
 'Churn_MultipleLines_max_kfold',
 'Churn_MultipleLines_median_kfold',
 'Churn_MultipleLines_min_kfold',
 'Churn_MultipleLines_nunique_kfold',
 'Churn_MultipleLines_q1_kfold',
 'Churn_OnlineBackup_max_kfold',
 'Churn_OnlineBackup_median_kfold',
 'Churn_OnlineBackup_min_kfold',
 'Churn_OnlineBackup_nunique_kfol

In [84]:
TarEnc_train['Churn_Contract_max_kfold']

0       1.0
1       1.0
3       1.0
4       1.0
6       1.0
       ... 
7038    1.0
7039    1.0
7040    1.0
7041    1.0
7042    1.0
Name: Churn_Contract_max_kfold, Length: 5282, dtype: float64

- 方差分析

&emsp;&emsp;接下来进行目标编码的标签关联度特征筛选，这里需要注意的是目标编码衍生出来的特征都是间接统计出来的结果，因此从数值层面上来看，衍生特征和标签的关联度或者和标签分布的一致性都会比较若，但这并不代表这些特征在建模过程中无法提供有效信息。简而言之，目标编码的衍生特征往往在标签关联度指标上会表现较弱（这点在特征衍生的实验中也有所体现），因此在进行特征筛选时可以略微放宽条件。

> 换个角度来说，之所以要进行交叉统计，也是为了让衍生特征的分布和标签分布尽可能有差异，以免标签信息泄露。

In [92]:
f_classif_p = f_classif(TarEnc_train[TarEnc_cols], y_train)[1]

In [93]:
f_classif_p

array([0.50836936, 0.50860777, 0.74039121, 0.64169298, 0.631999  ,
       0.64276059, 0.63962705, 0.63962705, 0.63711407, 0.64169298,
       0.50836936, 0.50857497, 0.99505711, 0.99174886, 0.98163218,
       0.9784985 , 0.97469648, 0.9784985 , 0.97311405, 0.97311405,
       0.97375077, 0.9784985 , 0.99505711, 0.99028738, 0.25340194,
       0.24648371, 0.14336331, 0.23264556, 0.23170547, 0.23264556,
       0.23504793, 0.23504793, 0.23383319, 0.23264556, 0.25340194,
       0.24632502, 0.99377509, 0.98257853, 0.93897658, 0.91830167,
       0.99181159, 0.91830167, 0.92790576, 0.92790576, 0.95193738,
       0.91830167, 0.99377509, 0.9581883 , 0.9012264 , 0.90001067,
       0.88861107, 0.88656073, 0.89483684, 0.8940054 , 0.8940054 ,
       0.89157671, 0.9012264 , 0.90406957, 0.48632285, 0.49012097,
       0.76545307, 0.59605164, 0.61673341, 0.60450688, 0.60663927,
       0.60663927, 0.60976087, 0.59605164, 0.48632285, 0.49073228,
       0.16223773, 0.10830268, 0.09080537, 0.41346215, 0.05220

能够大概看出，整体p值较高，因此这里在进行特征筛选时，可以稍微放宽显著性水平，由原来的0.01改为0.05。筛选过程如下：

In [94]:
f_classif_TarEnc_cols = []

for pValue, colname in zip(f_classif_p, TarEnc_cols):
    if pValue < 0.05:
        f_classif_TarEnc_cols.append(colname)

print(len(f_classif_TarEnc_cols))

28


In [95]:
f_classif_TarEnc_cols

['Churn_cv_OnlineBackup_kfold',
 'Churn_PaymentMethod_q2_kfold',
 'Churn_dive1_Churn_PaymentMethod_mean_kfold',
 'Churn_gap_PaymentMethod_kfold',
 'Churn_tenure_year_2015_mean_kfold',
 'Churn_tenure_year_2015_var_kfold',
 'Churn_tenure_year_2015_count_kfold',
 'Churn_tenure_year_2015_q2_kfold',
 'Churn_dive1_Churn_tenure_year_2015_mean_kfold',
 'Churn_dive2_Churn_tenure_year_2015_median_kfold',
 'Churn_minus1_Churn_tenure_year_2015_mean_kfold',
 'Churn_minus2_Churn_tenure_year_2015_mean_kfold',
 'Churn_norm_tenure_year_2015_kfold',
 'Churn_gap_tenure_year_2015_kfold',
 'Churn_mag1_tenure_year_2015_kfold',
 'Churn_cv_tenure_year_2015_kfold',
 'Churn_tenure_year_2019_mean_kfold',
 'Churn_tenure_year_2019_var_kfold',
 'Churn_tenure_year_2019_count_kfold',
 'Churn_tenure_year_2019_q2_kfold',
 'Churn_dive1_Churn_tenure_year_2019_mean_kfold',
 'Churn_dive2_Churn_tenure_year_2019_median_kfold',
 'Churn_minus1_Churn_tenure_year_2019_mean_kfold',
 'Churn_minus2_Churn_tenure_year_2019_mean_kfold

最终筛选出28个特征。

- 互信息法

&emsp;&emsp;接下来继续进行互信息法的特征筛选。这里有两点需要注意，其一是因为目标编码衍生的特征和标签关联度较弱，因此互信息计算结果整体数值较小，建议加上随机数种子以确保结果可以重复。其二则是这里也可以考虑放宽筛选条件，但由于互信息的筛选阈值是基于均值制定的，无论是否放宽筛选条件，互信息的阈值总是能一定程度确保最后筛选出来的特征数量（严重左偏除外）的，在方差分析仅筛选出了28个特征并且最终取交集的情况下，互信息无论是否放宽筛选条件，对最终的结果都不一定会有太大影响。

In [96]:
MI = mutual_info_classif(TarEnc_train[TarEnc_cols], y_train, random_state=22)
MI

array([6.98293019e-03, 1.84991056e-03, 7.22214150e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.19093239e-02, 0.00000000e+00,
       0.00000000e+00, 8.70209790e-03, 8.62784724e-03, 0.00000000e+00,
       1.96376184e-04, 1.19869684e-03, 1.01178922e-03, 3.64501811e-03,
       0.00000000e+00, 1.30463718e-02, 4.26812157e-03, 0.00000000e+00,
       4.06911734e-04, 0.00000000e+00, 4.78756641e-03, 4.29296654e-03,
       0.00000000e+00, 1.84902839e-03, 1.59428331e-03, 0.00000000e+00,
       8.07788762e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 9.51313838e-03, 5.36731696e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.05686105e-02, 2.08190928e-03,
       3.90446091e-03, 9.76629156e-03, 0.00000000e+00, 2.91048190e-03,
       9.29503574e-03, 4.00946841e-03, 4.82866803e-03, 4.17939559e-03,
       0.00000000e+00, 0.00000000e+00, 3.23634538e-03, 0.00000000e+00,
       2.75558196e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [97]:
len(MI)

456

然后同样选取mean\*0.01作为阈值进行特征筛选：

In [98]:
MI.mean()

0.0025913521951452675

In [99]:
MI_threshold = MI.mean() * 0.01
MI_threshold

2.5913521951452675e-05

In [100]:
MI_TarEnc_cols = []

for MIvalue, colname in zip(MI, TarEnc_cols):
    if MIvalue > MI_threshold:
        MI_TarEnc_cols.append(colname)

print(len(MI_TarEnc_cols))        

232


最后将两种方法挑选出来的特征取交集，并得到最终筛选出来的特征TarEnc_cols_select：

In [101]:
TarEnc_cols_select = list(set(f_classif_TarEnc_cols) & set(MI_TarEnc_cols))
len(TarEnc_cols_select)

15

In [102]:
TarEnc_cols_select

['Churn_mag1_tenure_year_2015_kfold',
 'Churn_gap_tenure_year_2015_kfold',
 'Churn_tenure_year_2015_q2_kfold',
 'Churn_minus2_Churn_tenure_year_2019_mean_kfold',
 'Churn_norm_tenure_year_2019_kfold',
 'Churn_tenure_year_2019_q2_kfold',
 'Churn_dive1_Churn_PaymentMethod_mean_kfold',
 'Churn_norm_tenure_year_2015_kfold',
 'Churn_cv_OnlineBackup_kfold',
 'Churn_tenure_year_2015_count_kfold',
 'Churn_minus2_Churn_tenure_year_2015_mean_kfold',
 'Churn_tenure_year_2015_var_kfold',
 'Churn_PaymentMethod_q2_kfold',
 'Churn_tenure_year_2019_mean_kfold',
 'Churn_dive2_Churn_tenure_year_2019_median_kfold']

经过筛选，目标编码最终输出了15个有效特征。

- 写入本地

&emsp;&emsp;同样，我们将上述衍生特征写入本地：

In [104]:
TarEnc_train[TarEnc_cols_select].reset_index(drop=True)

,Churn_mag1_tenure_year_2015_kfold,Churn_gap_tenure_year_2015_kfold,Churn_tenure_year_2015_q2_kfold,Churn_minus2_Churn_tenure_year_2019_mean_kfold,Churn_norm_tenure_year_2019_kfold,Churn_tenure_year_2019_q2_kfold,Churn_dive1_Churn_PaymentMethod_mean_kfold,Churn_norm_tenure_year_2015_kfold,Churn_cv_OnlineBackup_kfold,Churn_tenure_year_2015_count_kfold,Churn_minus2_Churn_tenure_year_2015_mean_kfold,Churn_tenure_year_2015_var_kfold,Churn_PaymentMethod_q2_kfold,Churn_tenure_year_2019_mean_kfold,Churn_dive2_Churn_tenure_year_2019_median_kfold
0,-0.281333,1.0,1.0,0.518939,1.038210,1.0,4.377378,-0.625575,1.928698,3750.0,-0.281333,0.202239,1.0,0.481061,100000.0
1,-0.281333,1.0,1.0,-0.167240,-0.448048,0.0,14.874910,-0.625575,1.226637,3750.0,-0.281333,0.202239,0.0,0.167240,0.0
2,-0.275053,1.0,1.0,-0.164835,-0.444173,0.0,0.000000,-0.615868,1.263923,3752.0,-0.275053,0.199452,0.0,0.164835,0.0
3,-0.281333,1.0,1.0,0.518939,1.038210,1.0,4.377378,-0.625575,1.226637,3750.0,-0.281333,0.202239,1.0,0.481061,100000.0
4,-0.276829,1.0,1.0,-0.168720,-0.450426,0.0,6.663735,-0.618610,1.965886,3746.0,-0.276829,0.200248,0.0,0.168720,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,-0.281333,1.0,1.0,-0.167240,-0.448048,0.0,14.874910,-0.625575,1.226637,3750.0,-0.281333,0.202239,0.0,0.167240,0.0
5278,-0.276829,1.0,1.0,-0.168720,-0.450426,0.0,6.663735,-0.618610,1.965886,3746.0,-0.276829,0.200248,0.0,0.168720,0.0
5279,-0.276829,1.0,1.0,0.526840,1.054777,1.0,4.428914,-0.618610,1.227203,3746.0,-0.276829,0.200248,1.0,0.473160,100000.0
5280,-0.278270,1.0,1.0,0.524427,1.049687,1.0,15.949152,-0.620838,1.245903,3723.0,-0.278270,0.200890,0.0,0.475573,100000.0


In [105]:
TarEnc_train[TarEnc_cols_select].reset_index(drop=True).to_csv('featuresCreation/X_train_TarEnc.csv', index=False)
TarEnc_test[TarEnc_cols_select].reset_index(drop=True).to_csv('featuresCreation/X_test_TarEnc.csv', index=False)

## 六、NLP特征衍生与筛选

&emsp;&emsp;NLP特征衍生无论是特征衍生过程还是特征筛选过程，都基本上和分组统计特征过程类似，衍生过程需要采用更强的KeyCol以增强特征衍生效果，而筛选过程则是将衍生特征视作连续变量，采用0值方差过滤、方差分析以及互信息法进行特征筛选。

### 1.NLP特征衍生过程

&emsp;&emsp;NLP特征衍生分为两部分执行，首先是围绕较强原始特征的IT-IDF特征衍生，然后是以这些较强原始特征为KeyCol来进行分组CountVectorizer和IT-IDF计算：

In [96]:
NLP_Group_Stat?

Signature:
NLP_Group_Stat(
    X_train,
    X_test,
    col_cat,
    keyCol=None,
    tfidf=True,
    countVec=True,
)
Docstring:
NLP特征衍生函数

:param X_train: 训练集特征
:param X_test: 测试集特征
:param col_cat: 参与衍生的离散型变量，只能带入多个列
:param keyCol: 分组参考的关键变量，输入字符串时代表按照单独列分组，输入list代表按照多个列进行分组
:param tfidf: 是否进行tfidf计算  
:param countVec: 是否进行CountVectorizer计算

:return：NLP特征衍生后的新特征和新特征的名称
File:      d:\work\jupyter\telco\features_creation.py
Type:      function


In [106]:
# 关键变量
col_temp = keycol.copy()

# 单变量if-idf计算
NLP_train, NLP_test, colNames_train_new, colNames_test_new = NLP_Group_Stat(train_temp_OE, 
                                                                            test_temp_OE, 
                                                                            col_temp)

# 以强原始特征作为keycol进行分组NLP特征衍生
for i in range(len(col_temp)):
    keyCol = col_temp.pop(i)
    features_train1, features_test1, colNames_train, colNames_test = NLP_Group_Stat(train_temp_OE, 
                                                                                    test_temp_OE, 
                                                                                    col_temp+cat_rest,
                                                                                    keyCol)
    
    NLP_train = pd.concat([NLP_train, features_train1],axis=1)
    NLP_test = pd.concat([NLP_test, features_test1],axis=1)
    
    col_temp = keycol.copy()

In [107]:
NLP_train.head()

,tenure_year_2015_tfidf,PaymentMethod_tfidf,PaperlessBilling_tfidf,tenure_year_2014_tfidf,tenure_year_2016_tfidf,InternetService_tfidf,Partner_tfidf,SeniorCitizen_tfidf,tenure_month_12_tfidf,OnlineBackup_tfidf,...,tenure_month_3_StreamingMovies_tfidf,tenure_month_4_StreamingMovies_tfidf,tenure_month_5_StreamingMovies_tfidf,tenure_month_6_StreamingMovies_tfidf,tenure_month_7_StreamingMovies_tfidf,tenure_month_8_StreamingMovies_tfidf,tenure_month_9_StreamingMovies_tfidf,tenure_month_10_StreamingMovies_tfidf,tenure_month_11_StreamingMovies_tfidf,tenure_quarter_3_StreamingMovies_tfidf
0,0.0,0.000000,0.167181,0.284834,0.0,0.000000,0.190072,0.000000,0.0,0.349988,...,0.023954,0.021216,0.020395,0.018753,0.021490,0.019711,0.021216,0.018616,0.021216,0.062417
1,0.0,0.355046,0.217072,0.000000,0.0,0.203693,0.246794,0.000000,0.0,0.454433,...,0.022190,0.021528,0.021528,0.027489,0.025171,0.025171,0.029642,0.032788,0.035107,0.079983
2,0.0,0.575611,0.000000,0.000000,0.0,0.440312,0.000000,0.000000,0.0,0.245580,...,0.018701,0.014249,0.017588,0.015139,0.016030,0.016252,0.016252,0.020037,0.020482,0.048534
3,0.0,0.506508,0.206450,0.000000,0.0,0.000000,0.234717,0.000000,0.0,0.432196,...,0.022190,0.021528,0.021528,0.027489,0.025171,0.025171,0.029642,0.032788,0.035107,0.079983
4,0.0,0.527340,0.000000,0.000000,0.0,0.403387,0.000000,0.400861,0.0,0.224986,...,0.018701,0.014249,0.017588,0.015139,0.016030,0.016252,0.016252,0.020037,0.020482,0.048534


In [108]:
NLP_train.shape

(5282, 1771)

NLP特征衍生总共创建1771个特征。

### 2.NLP衍生特征的筛选过程

&emsp;&emsp;接下来继续进行特征筛选，同样也是方差过滤、方差分析和互信息法三部分：

- 方差过滤

&emsp;&emsp;首先是方差过滤，这里仍然采用0值方差过滤：

In [109]:
sel = VarianceThreshold()
sel.fit(NLP_train)

VarianceThreshold()

In [110]:
NLP_cols = list(NLP_train.columns[sel.variances_ > 0])
len(NLP_cols)

1771

&emsp;&emsp;能够看出，NLP衍生特征中并不存在0方差的特征，主要原因也是组内元素个数的统计和TF-IDF的计算不太可能出现完全相同的结果。

- 方差分析

&emsp;&emsp;然后进一步进行方差分析：

In [111]:
f_classif_p = f_classif(NLP_train[NLP_cols], y_train)[1]

In [112]:
f_classif_p

array([4.53400235e-12, 4.44310685e-47, 8.16284694e-77, ...,
       3.18557233e-20, 1.74233920e-27, 1.79119043e-52])

这里仍然选取0.01为阈值，进行特征筛选：

In [113]:
f_classif_NLP_cols = []

for pValue, colname in zip(f_classif_p, NLP_cols):
    if pValue < 0.01:
        f_classif_NLP_cols.append(colname)

print(len(f_classif_NLP_cols))

1744


最终筛选出1744个特征，大部分特征都经过了方差分析的筛选。

- 互信息法

&emsp;&emsp;接下来进行互信息法特征筛选：

In [114]:
MI = mutual_info_classif(NLP_train[NLP_cols], y_train, random_state=22)
MI

array([0.01609048, 0.06766434, 0.07854761, ..., 0.03468068, 0.03966413,
       0.02046427])

In [115]:
len(MI)

1771

然后同样选取mean\*0.1作为阈值进行特征筛选：

In [116]:
MI.mean()

0.031270008838771964

In [117]:
MI_threshold = MI.mean() * 0.1
MI_threshold

0.0031270008838771967

In [118]:
MI_NLP_cols = []

for MIvalue, colname in zip(MI, NLP_cols):
    if MIvalue > MI_threshold:
        MI_NLP_cols.append(colname)

print(len(MI_NLP_cols))        

1603


互信息法挑选出1603个特征。最后将两种方法挑选出来的特征取交集，并得到最终筛选出来的特征NLP_cols_select：

In [119]:
NLP_cols_select = list(set(f_classif_NLP_cols) & set(MI_NLP_cols))
len(NLP_cols_select)

1576

最终筛选出1576个特征。而至此，我们也完成了全部特征衍生及初步特征筛选工作。

- 写入本地

&emsp;&emsp;同样，我们将上述衍生特征写入本地：

In [120]:
NLP_train[NLP_cols_select]

,PaymentMethod_tenure_year_2019_cntv,tenure_month_6_PaperlessBilling_cntv,tenure_year_2020_PaymentMethod_tfidf,tenure_year_2014_DeviceProtection_cntv,tenure_month_1_PaymentMethod_cntv,tenure_quarter_3_DeviceProtection_cntv,tenure_month_10_tenure_quarter_1_tfidf,TechSupport_PaperlessBilling_cntv,tenure_year_2017_StreamingMovies_tfidf,PaymentMethod_PaperlessBilling_tfidf,...,DeviceProtection_InternetService_cntv,PhoneService_tenure_year_2019_cntv,tenure_month_3_PaperlessBilling_tfidf,tenure_year_2019_PaymentMethod_tfidf,DeviceProtection_tenure_year_2015_tfidf,tenure_month_3_StreamingMovies_tfidf,OnlineSecurity_tenure_month_12_tfidf,tenure_year_2018_tenure_month_1_tfidf,tenure_year_2017_tenure_month_12_tfidf,MultipleLines_tenure_quarter_4_cntv
0,4988.0,238.0,0.000612,211.0,201.0,527.0,0.045297,2267.0,0.032851,0.459855,...,1602.0,3284.0,0.023311,0.041802,0.256291,0.023954,0.234865,0.043415,0.035195,3725.0
1,3346.0,238.0,0.000000,211.0,123.0,527.0,0.045297,2267.0,0.042062,0.459855,...,2034.0,1473.0,0.023311,0.149733,0.256291,0.022190,0.234865,0.043415,0.035195,1200.0
2,3346.0,133.0,0.002503,208.0,81.0,218.0,0.045297,1932.0,0.028497,0.477412,...,1123.0,1473.0,0.020250,0.162834,0.256291,0.018701,0.234865,0.043415,0.035195,3725.0
3,3346.0,238.0,0.002503,649.0,81.0,412.0,0.000000,2267.0,0.042062,0.459855,...,1602.0,1473.0,0.023311,0.162834,0.256291,0.022190,0.234865,0.043415,0.035195,3725.0
4,3346.0,133.0,0.002503,208.0,81.0,218.0,0.045297,1932.0,0.028497,0.477412,...,1123.0,1473.0,0.020250,0.162834,0.256291,0.018701,0.234865,0.043415,0.035195,3725.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,4988.0,238.0,0.000000,649.0,123.0,412.0,0.045297,2267.0,0.032851,0.459855,...,2034.0,3284.0,0.023311,0.149733,0.292573,0.023954,0.234865,0.043415,0.035195,3725.0
5278,4988.0,238.0,0.000000,211.0,123.0,527.0,0.045297,2267.0,0.042062,0.459855,...,1602.0,3284.0,0.023311,0.149733,0.256291,0.022190,0.234865,0.043415,0.035195,3725.0
5279,4988.0,238.0,0.000000,649.0,123.0,412.0,0.045297,2267.0,0.032851,0.459855,...,2034.0,3284.0,0.023311,0.149733,0.256291,0.023954,0.234865,0.043415,0.035195,3725.0
5280,4988.0,133.0,0.000320,208.0,178.0,218.0,0.045297,1932.0,0.028497,0.477412,...,1123.0,3284.0,0.020250,0.042692,0.256291,0.018701,0.234865,0.043415,0.035195,1200.0


In [121]:
NLP_train[NLP_cols_select].to_csv('featuresCreation/X_train_NLP.csv', index=False)
NLP_test[NLP_cols_select].to_csv('featuresCreation/X_test_NLP.csv', index=False)

## 七、关于平衡效率与效果的相关讨论

&emsp;&emsp;需要注意的是，关于上述特征衍生与特征筛选的流程，并不是完全固定的，也并不一定是唯一最优的策略，总体来看仍然是希望能够在保证效率的情况下，尽可能创建更好的结果，也就是做到效果和效率的平衡。

- 上行拓展

&emsp;&emsp;当然，如果算力足够，也可以在每个阶段进行更大规模的特征衍生，如多变量交叉组合、三阶多项式、双变量交叉组合分组衍生等等，同时也完全可以在特征初筛阶段放宽筛选要求，例如都以p值0.05作为阈值进行筛选、或者选择互信息的均值作为阈值进行筛选等。当然，这样一来，特征筛选的压力就会转移到下个阶段、也就是带入模型进行特征筛选的阶段。这样一来或许能创建更多更好的特征，但也将极大的增加算力的消耗。

- 下行拓展

&emsp;&emsp;另一方面，如果算力紧张，也可以略微精简上述流程，例如只针对效果最好的连续变量进行多项式衍生、提高分组统计特征衍生KeyCol的筛选标准等等；而在特征筛选阶段，也可以稍微提高特征筛选阈值，以减少下个阶段模型筛选的算力压力。当然，代价就是可能会损失一些对建模有帮助的高价值特征。

&emsp;&emsp;不过就目前数据集的数据量规模来看，当前的特征衍生和特征筛选流程是能够较好的兼顾到效果和效率。上述全部代码运行耗时差不多在10分钟内，没有必要再精简流程。

## 八、衍生特征读取过程

&emsp;&emsp;如果是从本部分开始运行代码，则可以通过如下过程进行衍生特征读取与全数据拼接。

- 衍生特征读取

In [126]:
X_train_seq = pd.read_csv('featuresCreation/X_train_seq.csv')
X_train_CrossComb = pd.read_csv('featuresCreation/X_train_CrossComb.csv')
X_train_Poly = pd.read_csv('featuresCreation/X_train_Poly.csv')
X_train_GroupStat = pd.read_csv('featuresCreation/X_train_GroupStat.csv')
X_train_TarEnc = pd.read_csv('featuresCreation/X_train_TarEnc.csv')
X_train_NLP = pd.read_csv('featuresCreation/X_train_NLP.csv')

In [132]:
X_test_seq = pd.read_csv('featuresCreation/X_test_seq.csv')
X_test_CrossComb = pd.read_csv('featuresCreation/X_test_CrossComb.csv')
X_test_Poly = pd.read_csv('featuresCreation/X_test_Poly.csv')
X_test_GroupStat = pd.read_csv('featuresCreation/X_test_GroupStat.csv')
X_test_TarEnc = pd.read_csv('featuresCreation/X_test_TarEnc.csv')
X_test_NLP = pd.read_csv('featuresCreation/X_test_NLP.csv')

- 数据集拼接

In [129]:
features_train_new = pd.concat([X_train_seq, 
                                X_train_CrossComb, 
                                X_train_Poly, 
                                X_train_GroupStat, 
                                X_train_TarEnc, 
                                X_train_NLP], axis=1)

In [133]:
features_test_new = pd.concat([X_test_seq, 
                               X_test_CrossComb, 
                               X_test_Poly, 
                               X_test_GroupStat, 
                               X_test_TarEnc, 
                               X_test_NLP], axis=1)

In [130]:
features_train_new.shape

(5282, 13992)

In [134]:
features_test_new.shape

(1761, 13992)

In [131]:
assert features_train_new.shape[1] == (X_train_seq.shape[1] + 
                                       X_train_CrossComb.shape[1] + 
                                       X_train_Poly.shape[1] + 
                                       X_train_GroupStat.shape[1] + 
                                       X_train_TarEnc.shape[1] + 
                                       X_train_NLP.shape[1])

In [135]:
assert features_test_new.shape[1] == (X_test_seq.shape[1] + 
                                      X_test_CrossComb.shape[1] + 
                                      X_test_Poly.shape[1] + 
                                      X_test_GroupStat.shape[1] + 
                                      X_test_TarEnc.shape[1] + 
                                      X_test_NLP.shape[1])

In [137]:
features_train_new.head()

,tenure_year_2014,tenure_year_2015,tenure_year_2016,tenure_year_2017,tenure_year_2018,tenure_year_2019,tenure_year_2020,tenure_month_1,tenure_month_2,tenure_month_3,...,DeviceProtection_InternetService_cntv,PhoneService_tenure_year_2019_cntv,tenure_month_3_PaperlessBilling_tfidf,tenure_year_2019_PaymentMethod_tfidf,DeviceProtection_tenure_year_2015_tfidf,tenure_month_3_StreamingMovies_tfidf,OnlineSecurity_tenure_month_12_tfidf,tenure_year_2018_tenure_month_1_tfidf,tenure_year_2017_tenure_month_12_tfidf,MultipleLines_tenure_quarter_4_cntv
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1602.0,3284.0,0.023311,0.041802,0.256291,0.023954,0.234865,0.043415,0.035195,3725.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2034.0,1473.0,0.023311,0.149733,0.256291,0.022190,0.234865,0.043415,0.035195,1200.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1123.0,1473.0,0.020250,0.162834,0.256291,0.018701,0.234865,0.043415,0.035195,3725.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1602.0,1473.0,0.023311,0.162834,0.256291,0.022190,0.234865,0.043415,0.035195,3725.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1123.0,1473.0,0.020250,0.162834,0.256291,0.018701,0.234865,0.043415,0.035195,3725.0


- 写入完整的衍生特征数据集

In [138]:
features_train_new.to_csv('featuresCreation/features_train_new.csv', index=False)

In [139]:
features_test_new.to_csv('featuresCreation/features_test_new.csv', index=False)

后续的各项建模工作将统一上述数据集作为衍生数据集。